In [41]:
import sys
import re
sys.path.append(r'C:/Users/shyam.p/Desktop/PRO/')
from  lib.initiate_ssh_connection import SSHClient

In [183]:
def  interface_status(obj):
    status = dict()
    m = avl_int(obj)
    for i in m:
        f = obj.run_execmcd(f"ethtool {i} | grep 'Link detected' | awk '{{print $3}}'")
        status[i] = f.strip()
    for i in list(status.keys()):
        if status[i] == '':
            status.pop(i)
    return status


In [167]:
interface_status(obj)

{}

In [168]:
x = (obj.run_execmcd("ip route | grep default | awk '{print $3}'")).strip()


In [184]:
def troubleshoot(obj):
    count = 1
    report = ''
    issu_int = []
    # Get the default gateway
    gate_way = (obj.run_execmcd("ip route | grep default | awk '{print $3}'")).strip()
    
    if not gate_way:
        return "Error: Default gateway not found."

    # Ping the default gateway
    ping_out = obj.run_execmcd(f'ping -c 4 {gate_way}')
    
    # Parse ping output
    pattern = re.compile(r'(\d+)% packet loss')
    match = pattern.search(ping_out)

    if match:
            packet_loss = int(match.group(1))

            if packet_loss == 0:
                report += f'{str(count)} --> Gateway {gate_way} is reachable\n'
                count += 1
            else:
                report += f'{str(count)} --> Gateway {gate_way} is not reachable\n'
                count += 1
    else:
         report += f"Error: Unable to parse packet loss information for {gate_way}\n"
        
    interfaces = interface_status(obj)
    for i in list(dict(interfaces).keys()):
        if interfaces[i] == 'no':
            issu_int.append(i)
    if issu_int[0] != '':
        report =  report + f'{str(count)} --> ' + 'Down Interface are' + '\n'
        count += count
        for i in issu_int:
            report =  report + '\t' + i +'\n'
    else:
        report = report + f'{str(count)} -->' + 'All Interface are up'

    return report

In [185]:
print(troubleshoot(obj))

1 --> Gateway 192.168.107.1 is reachable
2 --> Down Interface are
	eth2



In [156]:
### intractive shell
import paramiko

In [158]:
def spawn_ssh_terminal(host, username, password, port=22):
    # Create an SSH client
    ssh = paramiko.SSHClient()

    try:
        # Automatically add the server's host key (this is insecure, see comments below)
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Connect to the remote server
        ssh.connect(host, port=port, username=username, password=password)

        # Open an interactive SSH session (pseudo-terminal)
        channel = ssh.invoke_shell()

        # You can interact with the channel, send and receive commands
        while True:
            command = input("Enter command (or 'exit' to quit): ")
            if command.lower() == 'exit':
                break

            # Send the command to the server
            channel.send(command + '\n')

            # Receive and print the output
            output = channel.recv(1024).decode('utf-8')
            print(output, end='')

    finally:
        # Close the SSH connection
        ssh.close()

# Replace these with your actual SSH server details
host = '192.168.107.95'
username = 'root'
password = 'ewn@123'

spawn_ssh_terminal(host, username, password)

[root @ 2900aPoE/]# ls
bin    etc    lib    mnt    sbin   tmp    var
dev    home   lib32  proc   sys    usr
[root @ 2900aPoE/]# q
-sh: q: command not found
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# q
-sh: q: command not found
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 
[root @ 2900aPoE/]# 

In [ ]:
################################### Database Testing #########################################################

In [44]:
############# NEED TO FUNCTION ###################################################
import mysql.connector


# Replace these values with your actual database credentials
def sql_connect():
    host = "192.168.109.137"
    port = 3306
    user = "root"
    password = "password"
    database = "DEVICE_TRACKER"

    # Establish a connection to the database
    connection = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )
    return connection

In [49]:


def get_dev_details():
    dev = {}
    connection = sql_connect()
    try:
    # Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Define the SQL query to retrieve IP and TAG from DEVICES table
        query = r"SELECT ip, tags FROM field"

        # Execute the query
        cursor.execute(query)

        # Fetch all the rows
        rows = cursor.fetchall()

        # Print the results
        for row in rows:
            ip, tag = row
            dev[ip] = tag

    except:
        pass
  #  cursor.close()
    connection.close()
    return dev

def get_pass_details(ip):
    connection = sql_connect()
    cursor = connection.cursor()
    query =  r"SELECT username, password FROM field WHERE IP = %s"
    cursor.execute(query, (ip,))
    cred = cursor.fetchone()
    connection.close()
    return cred




In [51]:
l = get_dev_details()
l

{'192.16.107.71': 'BGP_SERVER',
 '192.16.2.1': 'Database ,Server',
 '192.168.107.109': 'SAMPLE-DUT',
 '192.168.107.20': 'Sample-Dut-2',
 '192.168.107.21': 'LAN-DEVICE',
 '192.168.107.237': 'E8K-Manu',
 '192.168.107.95': 'HA-DUT-1',
 '192.168.109.137': 'MSA-VM',
 '192.168.107.96': 'HA-DUT'}

In [97]:
l = get_pass_details('192.168.107.95')
l

('root', 'ewn@123')

In [134]:
import socket
####################################################### PING CHECk CODE##################################
def check_ping(devices):
    ping_status = dict()
    for ip, tag in devices.items():
        try:
            # Create a socket object and attempt to connect to the IP address on port 80
            sock = socket.create_connection((ip, 80), timeout=2)
            sock.close()
            ping_status[ip] = 'UP'
        except (socket.timeout, socket.error):
            ping_status[ip] = 'DOWN'
    return ping_status

check_ping(dev)


{'192.168.107.21': 'UP', '192.168.107.95': 'DOWN'}

In [129]:
################################################# Initiating SSH FOR STATUS CHECK ##############################

def initiate_ssh(dev):
    status_check = {}
    for ip, _ in dev.items():
        status = {}
        username, password = get_pass_details(ip)
        obj = SSHClient(ip,username,password)
        obj.connect()
        intefaces = obj.run_execmcd("ifconfig  | awk '/^[a-zA-Z]/{print $1}'")
        intefaces = intefaces.split('\n')
        intefaces.remove('')
        intefaces.remove('lo')
        for i in intefaces:
            interface = obj.run_execmcd(f"ethtool {i} | grep 'Link detected' | awk '{{print $3}}'")
            status[i] = interface.strip()
        status_check[ip] = status
        obj.close_connection()
    return status_check
################################################## Completed ###################################################

In [130]:
l = initiate_ssh(dev)
l

Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
Connected to 192.168.107.95
Connection to 192.168.107.95 closed.


{'192.168.107.21': {'eth0': 'yes', 'eth1': 'yes', 'eth2': 'yes'},
 '192.168.107.95': {'eth0': 'yes', 'eth1': 'yes', 'eth2': 'yes'}}

In [131]:
########################################### Ping Check #########################################################


{'192.168.107.21': {'eth0': 'yes', 'eth1': 'yes', 'eth2': 'yes'},
 '192.168.107.95': {'eth0': 'yes', 'eth1': 'yes', 'eth2': 'yes'}}

In [100]:
import socket

def check_ping(ip_address, port=80, timeout=2):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(timeout)
    result = sock.connect_ex((ip_address, port))
    sock.close()

    return result == 0  # 0 means the connection was successful

# Example usage
devices = {'192.168.107.21': 'Device1', '192.168.107.22': 'Device2'}
ping_status = {ip: 'UP' if check_ping(ip) else 'DOWN' for ip in devices}

for ip, status in ping_status.items():
    print(f"{devices[ip]} ({ip}) is {status}")


Device1 (192.168.107.21) is UP
Device2 (192.168.107.22) is DOWN


In [38]:
import paramiko

# Set your SSH parameters
hostname = '192.168.109.73'
port = 22
username = 'thinkpalm'
password = 'Think@123'

# Create an SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    # Connect to the remote server
    ssh.connect(hostname, port, username, password)

    # Execute the 'ifconfig' command
    ifconfig_command = "route -n"
    stdin, stdout, stderr = ssh.exec_command(ifconfig_command)
    print(str(stdout.read().decode(errors='replace')))
    # Print the output of the 'ifconfig' command
    #stdout = str(stdout.read().decode())
    #stdout = stdout.split('\n')
    #print(stdout)
    #stdout.remove('')
    ##   if i[-1] == ':':
      #      print(i[:-1])
finally:
    # Close the SSH connection
    ssh.close()


In [1]:
ignore_interface = ['cni-podman0:', 'fptun0:', 'ge1:', 'ge2:', 'ge3:', 'ge4:', 'ge5:', 'ge6:', 'ge7:', 'ge8:', 'lo:', 'sfp9:', 'sfp10:', 'veth707a8e75:', 'vnet1:', 'vnet3:','io','cni-podman0:', 'eth-int:', 'veth6128b75f:', 'vnet0:', 'vnet2:', '']

In [6]:
remove_inf = ['sfp10', 'veth707a8e75', 'vnet1', 'vnet3','io','cni-podman0', 'eth-int', 'veth6128b75f', 'vnet0', 'vnet2']

In [7]:
### Interface process

def procees_interface(inf):
    final_inf = []
    try:
        inf.remove('')
    except:
        pass
    for i in inf:
        if i[-1] == ':':
            final_inf.append(i[:-1])
        else:
            final_inf.append(i)
    for i in remove_inf:
        try:
            final_inf.remove(i)
        except:
            pass
    return final_inf

In [8]:
procees_interface(ignore_interface)

['fptun0', 'ge1', 'ge2', 'ge3', 'ge4', 'ge5', 'ge6', 'ge7', 'ge8', 'lo', 'sfp9', 'cni-podman0']


In [7]:
ignore_interface = ['cni-podman0', 'fptun0',  'lo',  'veth', 'vneth', 'vnet','io','cni-podman0', 'eth-int', 'vethb', 'vnet', 'vnet','cni-podman1','docker0']

def process_interface(inf):
    final_inf = []
    try:
        inf.remove('')
    except ValueError:
        pass

    for i in inf:
        if i[-1] == ':':
            final_inf.append(i[:-1])
        else:
            final_inf.append(i)

    # Remove interfaces based on substrings in ignore_interface
    final_inf = [interface for interface in final_inf if not any(ignore in interface for ignore in ignore_interface)]

    return final_inf

# Example usage:
input_interfaces = ['eth0', 'cni-podman0', 'vnet2', 'docker0', 'fptun0', 'veth707a8e75','vethb0e86d3','vnet1']
processed_interfaces = process_interface(input_interfaces)
print(processed_interfaces)


['eth0']
